# Traveling Salesman Problem

The traveling salesman problem requires finding the optimal route to be taken by a "traveling salesman" as they visit a set of cities.  The distances between cities are given, and the goal is to find the minimum distance to be traveled if we want to visit all the cities and return to the starting point.

In general, this falls in the category of NP-hard problems: in other words, there is no known polynomial time algorithm that will give the minimum distance to be traveled - any solution to this problem more or less requires enumerating all permutations, which is more than exponential in the number of cities.

Here we consider a simplified version of the problem, where cities are specified by their (x, y) coordinates, and the distance between cities is assumed to be the Euclidean distance.  This is unrealistic in practice since cities are normally connected by roads, and not all cities may be connected to all other cities, but this simple problem helps to illustrate one approach to solving such problems using Simulated Annealing.

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random

## Utility functions

We first define some utility functions that will be used by the optimization routine.

In [ ]:
# Generate random cities
def generate_cities(num_cities):
    return [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(num_cities)]

In [ ]:
# Calculate the total distance of a route
def calculate_distance(route, cities):
    return sum(np.sqrt((cities[route[i]][0] - cities[route[i-1]][0])**2 + 
                       (cities[route[i]][1] - cities[route[i-1]][1])**2) 
               for i in range(len(route)))

## Neighbour generation

The main point of the simulated annealing algorithm is to generate *neighbours* - solutions that are close to the existing solution, but different in some way, in order to evaluate whether the new solution is better than the existing one.  Here we use the approach that swapping two cities results in a new solution that is considered a *neighbour* of the existing one.

In [ ]:
# Generate a neighbour solution by swapping two cities
def get_neighbour(route):
    new_route = route.copy()
    i, j = random.sample(range(len(route)), 2)
    new_route[i], new_route[j] = new_route[j], new_route[i]
    return new_route

## Simulated Annealing

The actual annealing process involves iterating until some condition is satisfied: in this case we use a fixed number of iterations as the terminating condition.  You could also decide to continue until there is no improvement in the solution for some number of iterations, or the cost goes below a certain threshold, etc.

At each iteration, a new neighbour is generated.  The cost of this is calculated, and the decision to *accept* or *reject* the new solution is based on either actual improvement, or a *coin toss* - a random event whose probability decreases with iterations.  Eventually, the chance of accepting poorer solutions is so low that we settle to a solution, which hopefully is a good solution given the method used for searching through the neighbourhood.

In [ ]:
# Simulated annealing algorithm
def simulated_annealing(cities, initial_temp, cooling_rate, num_iterations):
    current_route = list(range(len(cities)))
    random.shuffle(current_route)
    current_distance = calculate_distance(current_route, cities)
    
    best_route = current_route.copy()
    best_distance = current_distance
    
    temp = initial_temp
    distances = [current_distance]
    best_routes = [best_route.copy()]
    best_distances = [best_distance]
    
    for i in range(num_iterations):
        neighbour_route = get_neighbour(current_route)
        neighbour_distance = calculate_distance(neighbour_route, cities)
        
        p = np.exp((current_distance - neighbour_distance) / temp)
        # print(f"Prob[{i} = {p}")
        if neighbour_distance < current_distance or random.random() < p:
            current_route = neighbour_route
            current_distance = neighbour_distance
            
            if current_distance < best_distance:
                best_route = current_route.copy()
                best_distance = current_distance
        
        temp *= cooling_rate
        distances.append(current_distance)
        best_routes.append(best_route.copy())
        best_distances.append(best_distance)

    return best_routes, best_distances, distances

## Animation

We would like to visualize this entire process as an animation, so we set up an update function that can be used for the `matplotlib` `FuncAnimation` method.  Note that here we actually process the entire simulated annealing in advance and then create the animation, but in practice it may be better to do it *live* since the evaluation of costs itself could be expensive.

In [ ]:
# Animation update function
def update(frame, cities, routes, distances, cur_dist, route_line, distance_line, cur_dist_line):
    route = routes[frame]
    coords = np.array([cities[i] for i in route + [route[0]]])
    route_line.set_data(coords[:, 0], coords[:, 1])
    distance_line.set_data(range(frame + 1), distances[:frame + 1])
    cur_dist_line.set_data(range(frame + 1), cur_dist[:frame + 1])
    return route_line, distance_line, cur_dist_line

## Actually running the code

Finally we collate the actual running of the algorithm into a single `main` function so that it can be used as a standalone script.  For a Jupyter notebook this is not needed, but is still generally a good way to organize your code.

In [ ]:
# Main function to run the algorithm and create the animation
def main():
    num_cities = 20
    initial_temp = 100
    cooling_rate = 0.995
    num_iterations = 100

    cities = generate_cities(num_cities)
    best_routes, best_distances, cur_dist = simulated_annealing(cities, initial_temp, cooling_rate, num_iterations)

    # Set up the figure and subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle("Simulated Annealing for Traveling Salesman Problem")

    # Route subplot
    ax1.set_xlim(0, 100)
    ax1.set_ylim(0, 100)
    ax1.set_title("Best Route")
    ax1.set_xlabel("X coordinate")
    ax1.set_ylabel("Y coordinate")
    route_line, = ax1.plot([], [], 'bo-')

    # Distance subplot
    ax2.set_xlim(0, num_iterations)
    ax2.set_ylim(min(best_distances) * 0.9, max(best_distances) * 1.1)
    ax2.set_title("Best Distance over Iterations")
    ax2.set_xlabel("Iteration")
    ax2.set_ylabel("Distance")
    distance_line, = ax2.plot([], [], 'r-')
    cur_dist_line, = ax2.plot([], [], 'g-')

    # Create the animation
    anim = FuncAnimation(fig, update, frames=range(0, num_iterations, 1), 
                         fargs=(cities, best_routes, best_distances, cur_dist, route_line, distance_line, cur_dist_line),
                         interval=50, blit=True, repeat=False)
    return anim
    # Use plt.show() if running as a script
    # plt.tight_layout()
    # plt.show()
    

In [ ]:
# In a script, we would have 
# if __name__ == '__main__':
#    main()
# Use the HTML animation for display inside a notebook
anim = main()
from IPython.display import HTML
HTML(anim.to_jshtml())
